In [32]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torchvision.transforms import Normalize
from ax import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting
import torch.nn as nn
import logging



class CNNTrainer:
    def __init__(self, train_data, val_data=None, batch_size=64, model=None):
        self.model = model
        # Load the training data
        self.train_loader = data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
        if val_data: 
            self.val_loader = data.DataLoader(train_data, batch_size=batch_size)
            
    def compute_input_size(self, params):
        # Add BatchNorm2d layer to standardize input data
        x = nn.BatchNorm2d(3)(torch.zeros([1,3,32,32]))
        pool1 = eval(params.get("pool1"))
        pool2 = eval(params.get("pool2"))        
        # Pass the input through the sequence of layers
        for layer in [
                nn.Conv2d(in_channels=3, 
                          out_channels=params.get('num_filters1'), 
                          kernel_size=params.get('filter_size1')),
                nn.ReLU(),
                pool1(kernel_size=params.get("kernel_pool1")),
                nn.Conv2d(in_channels=params.get('num_filters1'),
                          out_channels=params.get('num_filters2'),
                          kernel_size=params.get('filter_size2')),
                nn.ReLU(),
                pool2(kernel_size=params.get("kernel_pool2")),
                nn.Conv2d(in_channels=params.get('num_filters2'),
                          out_channels=params.get('num_filters3'),
                          kernel_size=params.get('filter_size3')),
                nn.ReLU(),
                nn.Flatten()]:
            x = layer(x)
        input_size = x.shape[0] * x.shape[1]
        return input_size

    def build_model(self, params):
        linear_input = self.compute_input_size(params) 
        # Define the CNN architecture based on the given parameters
        pool1 = eval(params.get("pool1"))
        pool2 = eval(params.get("pool2"))
        model = nn.Sequential(
            nn.BatchNorm2d(3),  # Add BatchNorm2d layer to standardize input data,
            nn.Conv2d(in_channels=3, 
                      out_channels=params.get('num_filters1'), 
                      kernel_size=params.get('filter_size1')),
            nn.ReLU(),
            pool1(kernel_size=params.get("kernel_pool1")),
            nn.Conv2d(in_channels=params.get('num_filters1'),
                      out_channels=params.get('num_filters2'),
                      kernel_size=params.get('filter_size2')),
            nn.ReLU(),
            pool2(kernel_size=params.get("kernel_pool2")),
            nn.Conv2d(in_channels=params.get('num_filters2'),
                      out_channels=params.get('num_filters3'),
                      kernel_size=params.get('filter_size3')),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(linear_input, 10)
        )
        return model
        
        
    def fit(self, model, epochs=1):
        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Train the model
        for epoch in range(epochs):
            print("Running epoch ", epoch)
            total, correct = 0,0
            for images, labels in self.train_loader:
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            accuracy = correct / total  
            print(f"ACC for epoch {epoch}: ", accuracy)
        return model
        
class CNNPredictor:
    def __init__(self, model):
        self.model = model
        
    def predict(self, test_data):
        # Evaluate the model on the validation set
        data_loader = data.DataLoader(test_data, batch_size=64)
        
        # Calculate validation accuracy
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in data_loader:
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        return labels, accuracy


class CNNOptimizer:
    def __init__(self, search_space, train_data, val_data, steps=20, epochs=1):
        self.epochs = epochs
        self.steps = steps
        self.search_space = search_space
        self.train_data = train_data
        self.val_data = val_data

    def evaluate_model(self, parameterization):
        try:
            print("Testing config", parameterization)
            trainer = CNNTrainer(self.train_data)
            model = trainer.build_model(parameterization) 
        except Exception as e:
            logging.error(e)
            return {'acc': 0} 
        
        print("CONFIG Valida")
        model = trainer.fit(model, epochs=self.epochs)
        predictor = CNNPredictor(model)
        _, accuracy = predictor.predict(self.val_data)
        print("ACC during eval", accuracy)
        # Return the validation accuracy as the objective value to optimize
        return {'acc': accuracy}

    def optimize(self):
        
        constraints = ["num_filters1 <= num_filters2",    
                       "num_filters2 <= num_filters3",   
                       "filter_size1 >= filter_size2",   
                       "filter_size2 >= filter_size3",   
                      ]    

        best_parameters, best_values, experiment, model = optimize(
            parameters=self.search_space,
            evaluation_function=self.evaluate_model,
            parameter_constraints=constraints,
            objective_name='acc',
            minimize=False,
            total_trials=self.steps
        )

        print('Best parameters:', best_parameters)
        print('Best validation accuracy:', best_values[0])
        
        return best_parameters, best_values, experiment, model

In [42]:
torch.optim.lr_scheduler.MultiplicativeLR?

Init signature:
torch.optim.lr_scheduler.MultiplicativeLR(
    optimizer,
    lr_lambda,
    last_epoch=-1,
    verbose=False,
)
Docstring:     
Multiply the learning rate of each parameter group by the factor given
in the specified function. When last_epoch=-1, sets initial lr as lr.

Args:
    optimizer (Optimizer): Wrapped optimizer.
    lr_lambda (function or list): A function which computes a multiplicative
        factor given an integer parameter epoch, or a list of such
        functions, one for each group in optimizer.param_groups.
    last_epoch (int): The index of last epoch. Default: -1.
    verbose (bool): If ``True``, prints a message to stdout for
        each update. Default: ``False``.

Example:
    >>> lmbda = lambda epoch: 0.95
    >>> scheduler = MultiplicativeLR(optimizer, lr_lambda=lmbda)
    >>> for epoch in range(100):
    >>>     train(...)
    >>>     validate(...)
    >>>     scheduler.step()
File:           ~/miniconda3/envs/mariner/lib/python3.8/site-packa

In [39]:
torch.optim.lr_scheduler.ExponentialLR?

Init signature:
torch.optim.lr_scheduler.ExponentialLR(
    optimizer,
    gamma,
    last_epoch=-1,
    verbose=False,
)
Docstring:     
Decays the learning rate of each parameter group by gamma every epoch.
When last_epoch=-1, sets initial lr as lr.

Args:
    optimizer (Optimizer): Wrapped optimizer.
    gamma (float): Multiplicative factor of learning rate decay.
    last_epoch (int): The index of last epoch. Default: -1.
    verbose (bool): If ``True``, prints a message to stdout for
        each update. Default: ``False``.
File:           ~/miniconda3/envs/mariner/lib/python3.8/site-packages/torch/optim/lr_scheduler.py
Type:           type
Subclasses:     


In [28]:
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Load the CIFAR10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())

# Define the search space

def split_train_val(data, ptrain=0.3, pval=0.2):
    train_mask = np.random.rand(len(data)) <= ptrain
    val_mask = np.random.rand(len(data)) >= (1-pval)
    index = np.array(range(len(data)))
    train_id, test_id = index[train_mask], index[val_mask]
    train = torch.utils.data.Subset(data, train_id)
    test = torch.utils.data.Subset(data, test_id)
    return train, test

train, val = split_train_val(train_dataset)

search_space = [
    {"name": "num_filters1", "type": "range", "bounds": [12, 32], "value_type":"int"},
    {"name": "filter_size1", "type": "range", "bounds": [3, 5], "value_type":"int"},
    {"name": "num_filters2", "type": "range", "bounds": [12, 32], "value_type":"int"},
    {"name": "filter_size2", "type": "range", "bounds": [3, 5], "value_type":"int"},
    {"name": "num_filters3", "type": "range", "bounds": [12, 32], "value_type":"int"},
    {"name": "filter_size3", "type": "range", "bounds": [3, 5], "value_type":"int"},
    {"name": "pool1", "type": "choice", "is_ordered": False,
     "values": ["nn.AvgPool2d", "nn.MaxPool2d"]},
    {"name": "pool2", "type": "choice", "is_ordered": False, 
     "values": ["nn.AvgPool2d", "nn.MaxPool2d"]},
    {"name": "kernel_pool1", "type": "range", "bounds": [2, 3], "value_type":"int"},
    {"name": "kernel_pool2", "type": "range", "bounds": [2, 3], "value_type":"int"}

]
# Initialize the CNNOptimizer
optimizer = CNNOptimizer(search_space, train, val, steps=20, epochs=2)

# Run the optimization
best_parameters, best_values, experiment, model = optimizer.optimize()


Files already downloaded and verified
Files already downloaded and verified


[INFO 03-24 18:01:19] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter pool1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/home/isac/miniconda3/envs/mariner/lib/python3.8/site-packages/ax/core/parameter.py:492: UserWarning:

`sort_values` is not specified for `ChoiceParameter` "pool1". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

[INFO 03-24 18:01:19] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter pool2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/home/isac/miniconda3/envs/mariner/lib/python3.8/site-packages/ax/core/parameter.py:492: UserWarning:

`sort_values` is not specified for `ChoiceParameter` 

Testing config {'num_filters1': 19, 'filter_size1': 3, 'num_filters2': 22, 'filter_size2': 3, 'num_filters3': 31, 'filter_size3': 3, 'kernel_pool1': 2, 'kernel_pool2': 2, 'pool1': 'nn.AvgPool2d', 'pool2': 'nn.MaxPool2d'}
CONFIG Valida
Running epoch  0
ACC for epoch 0:  0.3105741374769555
Running epoch  1
ACC for epoch 1:  0.4288253884645773


[INFO 03-24 18:01:32] ax.service.managed_loop: Running optimization trial 2...
ERROR:root:Calculated padded input size per channel: (2 x 2). Kernel size: (4 x 4). Kernel size can't be greater than actual input size
[INFO 03-24 18:01:32] ax.service.managed_loop: Running optimization trial 3...
ERROR:root:Calculated padded input size per channel: (3 x 3). Kernel size: (4 x 4). Kernel size can't be greater than actual input size
[INFO 03-24 18:01:32] ax.service.managed_loop: Running optimization trial 4...
ERROR:root:Calculated padded input size per channel: (2 x 2). Kernel size: (5 x 5). Kernel size can't be greater than actual input size
[INFO 03-24 18:01:32] ax.service.managed_loop: Running optimization trial 5...
ERROR:root:Calculated padded input size per channel: (1 x 1). Kernel size: (5 x 5). Kernel size can't be greater than actual input size
[INFO 03-24 18:01:32] ax.service.managed_loop: Running optimization trial 6...


ACC during eval 0.45773603559070686
Testing config {'num_filters1': 14, 'filter_size1': 5, 'num_filters2': 25, 'filter_size2': 5, 'num_filters3': 31, 'filter_size3': 4, 'kernel_pool1': 3, 'kernel_pool2': 2, 'pool1': 'nn.AvgPool2d', 'pool2': 'nn.AvgPool2d'}
Testing config {'num_filters1': 20, 'filter_size1': 5, 'num_filters2': 22, 'filter_size2': 4, 'num_filters3': 27, 'filter_size3': 4, 'kernel_pool1': 2, 'kernel_pool2': 3, 'pool1': 'nn.AvgPool2d', 'pool2': 'nn.AvgPool2d'}
Testing config {'num_filters1': 21, 'filter_size1': 5, 'num_filters2': 21, 'filter_size2': 5, 'num_filters3': 21, 'filter_size3': 5, 'kernel_pool1': 3, 'kernel_pool2': 2, 'pool1': 'nn.MaxPool2d', 'pool2': 'nn.AvgPool2d'}
Testing config {'num_filters1': 26, 'filter_size1': 5, 'num_filters2': 30, 'filter_size2': 5, 'num_filters3': 30, 'filter_size3': 5, 'kernel_pool1': 3, 'kernel_pool2': 3, 'pool1': 'nn.AvgPool2d', 'pool2': 'nn.MaxPool2d'}
Testing config {'num_filters1': 16, 'filter_size1': 5, 'num_filters2': 20, 'filt

...

ACC during eval 0.4889767671774592
Testing config {'num_filters1': 12, 'filter_size1': 5, 'num_filters2': 17, 'filter_size2': 5, 'num_filters3': 31, 'filter_size3': 3, 'kernel_pool1': 2, 'kernel_pool2': 3, 'pool1': 'nn.AvgPool2d', 'pool2': 'nn.MaxPool2d'}
CONFIG Valida
Running epoch  0
ACC for epoch 0:  0.2871345799315249
Running epoch  1
ACC for epoch 1:  0.41322096391888335


[INFO 03-24 18:03:49] ax.service.managed_loop: Running optimization trial 18...
ERROR:root:Calculated padded input size per channel: (2 x 2). Kernel size: (3 x 3). Kernel size can't be greater than actual input size
[INFO 03-24 18:03:49] ax.service.managed_loop: Running optimization trial 19...


ACC during eval 0.44092931290163123
Testing config {'num_filters1': 15, 'filter_size1': 4, 'num_filters2': 19, 'filter_size2': 3, 'num_filters3': 21, 'filter_size3': 3, 'kernel_pool1': 3, 'kernel_pool2': 3, 'pool1': 'nn.MaxPool2d', 'pool2': 'nn.MaxPool2d'}


ERROR:root:Calculated padded input size per channel: (2 x 2). Kernel size: (4 x 4). Kernel size can't be greater than actual input size
[INFO 03-24 18:03:49] ax.service.managed_loop: Running optimization trial 20...
ERROR:root:Calculated padded input size per channel: (2 x 2). Kernel size: (4 x 4). Kernel size can't be greater than actual input size


Testing config {'num_filters1': 15, 'filter_size1': 5, 'num_filters2': 16, 'filter_size2': 4, 'num_filters3': 28, 'filter_size3': 4, 'kernel_pool1': 3, 'kernel_pool2': 3, 'pool1': 'nn.MaxPool2d', 'pool2': 'nn.MaxPool2d'}
Testing config {'num_filters1': 13, 'filter_size1': 5, 'num_filters2': 15, 'filter_size2': 5, 'num_filters3': 31, 'filter_size3': 4, 'kernel_pool1': 3, 'kernel_pool2': 2, 'pool1': 'nn.AvgPool2d', 'pool2': 'nn.MaxPool2d'}
Best parameters: {'num_filters1': 17, 'filter_size1': 5, 'num_filters2': 21, 'filter_size2': 3, 'num_filters3': 29, 'filter_size3': 3, 'kernel_pool1': 2, 'kernel_pool2': 2, 'pool1': 'nn.MaxPool2d', 'pool2': 'nn.MaxPool2d'}
Best validation accuracy: {'acc': 0.4889767671774592}


In [29]:
best_parameters, best_values

({'num_filters1': 17,
  'filter_size1': 5,
  'num_filters2': 21,
  'filter_size2': 3,
  'num_filters3': 29,
  'filter_size3': 3,
  'kernel_pool1': 2,
  'kernel_pool2': 2,
  'pool1': 'nn.MaxPool2d',
  'pool2': 'nn.MaxPool2d'},
 ({'acc': 0.4889767671774592}, {'acc': {'acc': nan}}))

In [30]:
data = experiment.fetch_data()
data.df

,arm_name,metric_name,mean,sem,trial_index
0,0_0,acc,0.457736,NaN,0
1,1_0,acc,0.000000,NaN,1
2,2_0,acc,0.000000,NaN,2
3,3_0,acc,0.000000,NaN,3
4,4_0,acc,0.000000,NaN,4
5,5_0,acc,0.412259,NaN,5
6,6_0,acc,0.000000,NaN,6
7,7_0,acc,0.000000,NaN,7
8,8_0,acc,0.000000,NaN,8
9,9_0,acc,0.383292,NaN,9


In [33]:
train, val = split_train_val(train_dataset, ptrain=0.7, pval=0.3)

trainer = CNNTrainer(train)
model = trainer.build_model(best_parameters) 
model = trainer.fit(model, epochs=50)


Running epoch  0
ACC for epoch 0:  0.3952618094936527
Running epoch  1
ACC for epoch 1:  0.5121045510809177
Running epoch  2
ACC for epoch 2:  0.5552836869224791
Running epoch  3
ACC for epoch 3:  0.5884164771582372
Running epoch  4
ACC for epoch 4:  0.6179222199833041
Running epoch  5
ACC for epoch 5:  0.6377846224704223
Running epoch  6
ACC for epoch 6:  0.6565243674256599
Running epoch  7
ACC for epoch 7:  0.6652465528656553
Running epoch  8
ACC for epoch 8:  0.6737096634906014
Running epoch  9
ACC for epoch 9:  0.687526986959901
Running epoch  10
ACC for epoch 10:  0.6925933388986442
Running epoch  11
ACC for epoch 11:  0.6970551829356055
Running epoch  12
ACC for epoch 12:  0.7095483462390972
Running epoch  13
ACC for epoch 13:  0.7131753936497884
Running epoch  14
ACC for epoch 14:  0.7166009384265523
Running epoch  15
ACC for epoch 15:  0.7194219753015343
Running epoch  16
ACC for epoch 16:  0.7242868246063502
Running epoch  17
ACC for epoch 17:  0.7296122513601427
Running epoch

In [34]:
predictor = CNNPredictor(model)
_, accuracy = predictor.predict(val)

In [35]:
accuracy

0.7756534527709071

In [240]:
# unique, counts = np.unique(labels, return_counts=True)

In [237]:
counts

array([1472, 1493, 1496, 1443, 1500, 1504, 1443, 1547, 1453, 1463])